#
<span style="font-family: Times New Roman; font-size: 20px;">
<h1 align='center'> 
Tensorboard Implementation
</h1>

##
<span style="font-family: Times New Roman; font-size: 20px;">
<h2> 
Imports
</h2>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
import io,os,shutil
import numpy as np

##
<span style="font-family: Times New Roman; font-size: 20px;">
<h2> 
Function for Logging Images
</h2>

In [13]:
if os.path.exists("logs"):
    shutil.rmtree("logs")

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
 
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

def image_grid(images, class_names, class_labels):
    """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
    figure = plt.figure(figsize=(10,10))
    for i in range(images.shape[0]):
        plt.subplot(5, images.shape[0]//5, i+1, title=class_names[class_labels[i]])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
    
    return figure
    

In [14]:
# Function to log images to tensorboard
def log_images(model, images, class_names, logdir):
    file_writer = tf.summary.create_file_writer(logdir)
    with file_writer.as_default():
        predictions = model.predict(images)
        predicted_class = tf.argmax(predictions, axis=1)
        
        figure = image_grid(images, class_names, predicted_class)
        tf.summary.image("Test set predictions", plot_to_image(figure), step=0)
        

##
<span style="font-family: Times New Roman; font-size: 20px;">
<h2> 
Loading Model
</h2>

In [15]:
#Enter the path of the model here
model_path = ""
def load_model(model_path):
    model = tf.keras.models.load_model(model_path)
    return model

# model = load_model(model_path)

##
<span style="font-family: Times New Roman; font-size: 20px;">
<h2> 
Data Loader Function
</h2>

In [23]:
def data_loader(directory):
    import os
    import numpy as np
    import torch
    from keras.preprocessing import image
    for i, filename in enumerate(os.listdir(directory)):
        img = image.load_img(directory + filename, target_size=(256, 256))
        img = image.img_to_array(img)
        img = img.reshape((1,) + img.shape)
        if i == 0:
            images = img
        else:
            images = np.vstack((images, img))
    return torch.tensor(images)

##
<span style="font-family: Times New Roman; font-size: 20px;">
<h2> 
Loading the Test Data and Labels
</h2>

In [24]:
logdir = "logs/plots/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [25]:
test_data_j = data_loader("data/test/jaguar/")
labels_j = np.ones((test_data_j.shape[0], 1))
test_data_c = data_loader("data/test/cheetah/")
labels_c = np.zeros((test_data_c.shape[0], 1))

test_data = np.vstack((test_data_j, test_data_c))
labels = np.vstack((labels_j, labels_c))

np.random.seed(0)
np.random.shuffle(test_data)
np.random.shuffle(labels)

# Checking if the data is randomly shuffled correctly with the labels
print(test_data[:1].shape)
print(labels[:1])


(1, 256, 256, 3)
[[1.]]


In [18]:
%tensorboard --logdir logs/plots

UsageError: Line magic function `%tensorboard` not found.
